<a href="https://colab.research.google.com/github/rechavar/pyCovid_TechLabsProj/blob/Marco/TechLabs__finalProject_interactive_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TechLabs final project 
## (Problem 2 - Group 8)

---

This notebook represents the work in the final project of the Techlabs #codeathome course.
<br> <br>
Team members: Luca Wulfert, Laura Laks, Phillip Guske, Juliana Bedoya, Alejandro Echavarria and Marco Stoever

---

# Data Preparation

### libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Constants

In [0]:
confirmed_csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
death_csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
recovered_csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
countries_csv_url = 'https://raw.githubusercontent.com/rechavar/pyCovid_TechLabsProj/master/countries.csv'

### Data Exploration

---
GitHub repository: https://github.com/CSSEGISandData/COVID-19 
<br>
(c) Johns Hopkins University Center for Systems Science and Engineering <br> <br>
The datasets were also used by Kaggle for their weekly *COVID19 Global Forecasting* competition. <br>
(e.g. see: https://www.kaggle.com/c/covid19-global-forecasting-week-5)


---







In [0]:
df_confirmed = pd.read_csv(confirmed_csv_url)
df_death = pd.read_csv(death_csv_url)
df_recovered = pd.read_csv(recovered_csv_url)
df_countries = pd.read_csv(countries_csv_url)

In [53]:
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,996,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676,9216,9998,10582,11173,11831,12456,13036
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,562,584,609,634,663,678,712,726,736,750,766,773,782,789,795,803,820,832,842,850,856,868,872,876,880,898,916,933,946,948,949,964,969,981,989,998,1004,1029,1050,1076
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,2629,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648,4838,4997,5182,5369,5558,5723,5891,6067,6253,6442,6629,6821,7019,7201,7377,7542,7728,7918,8113,8306,8503,8697,8857,8997
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,713,717,717,723,723,731,738,738,743,743,743,745,745,747,748,750,751,751,752,752,754,755,755,758,760,761,761,761,761,761,761,762,762,762,762,762,763,763,763,763
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,24,24,24,25,25,25,25,26,27,27,27,27,30,35,35,35,36,36,36,43,43,45,45,45,45,48,48,48,48,50,52,52,58,60,61,69,70,70,71,74


In [54]:
df_death.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,33,36,36,40,42,43,47,50,57,58,60,64,68,72,85,90,95,104,106,109,115,120,122,127,132,136,153,168,169,173,178,187,193,205,216,218,219,220,227,235
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,26,26,26,27,27,27,27,28,28,30,30,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,32,32,33,33,33
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,375,384,392,402,407,415,419,425,432,437,444,450,453,459,463,465,470,476,483,488,494,502,507,515,522,529,536,542,548,555,561,568,575,582,592,600,609,617,623,630
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,36,37,37,37,37,40,40,40,40,41,42,42,43,44,45,45,46,46,47,47,48,48,48,48,49,49,49,51,51,51,51,51,51,51,51,51,51,51,51,51
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,4


In [55]:
df_recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,131,135,150,166,179,188,188,207,220,228,252,260,310,331,345,397,421,458,468,472,502,558,558,610,648,691,745,745,778,801,850,930,938,996,1040,1075,1097,1128,1138,1209
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,314,327,345,356,385,394,403,410,422,431,455,470,488,519,531,543,570,595,605,620,627,650,654,682,688,694,705,714,715,727,742,758,771,777,783,789,795,803,812,823
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1047,1099,1152,1204,1355,1408,1479,1508,1558,1651,1702,1779,1821,1872,1936,1998,2067,2197,2323,2467,2546,2678,2841,2998,3058,3158,3271,3409,3507,3625,3746,3968,4062,4256,4426,4784,4747,4918,5129,5277
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,235,248,282,309,333,344,344,344,385,398,423,468,468,472,493,499,514,521,526,537,545,550,550,568,576,596,604,615,617,624,628,639,639,652,653,653,663,676,676,681
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6,7,7,11,11,11,11,11,11,11,11,13,13,13,13,14,14,17,17,17,17,17,17,17,17,18,18,18,18,18,18


In [56]:
pd.set_option('display.max_rows', 300)

print(df_confirmed.iloc[:,1:2])
print(df_death.iloc[:,1:2])
print(df_recovered.iloc[:,1:2])

#Taiwan is written with an asterisk (*) at the end, we have to change this:
#We also want to have "USA" instead of "US":
#Congo (Brazzaville) and Congo (Kinshasa) are capitals of different countries!
#Brazzaville -> Republic of the Congo || Kinshasa -> Democratic Republic of the Congo

#We will change this in the data cleaning part

                       Country/Region
0                         Afghanistan
1                             Albania
2                             Algeria
3                             Andorra
4                              Angola
5                 Antigua and Barbuda
6                           Argentina
7                             Armenia
8                           Australia
9                           Australia
10                          Australia
11                          Australia
12                          Australia
13                          Australia
14                          Australia
15                          Australia
16                            Austria
17                         Azerbaijan
18                            Bahamas
19                            Bahrain
20                         Bangladesh
21                           Barbados
22                            Belarus
23                            Belgium
24                              Benin
25          

In [0]:
#reset max_rows displayed to default number (10):
pd.set_option('display.max_rows', 10)

In [0]:
all_dataframes = {'confirmed': df_confirmed,
                  'death': df_death,
                  'recovered': df_recovered,
                  'countries': df_countries}

### Data Cleaning

In [59]:
for key, df in all_dataframes.items():
  print('cleaning: {}'.format(key))

  if key == 'countries':
    continue

  df.drop(['Province/State', 'Lat', 'Long'], axis = 1, inplace = True)

  #Country renaming:
  df["Country/Region"].replace(["Taiwan*", "US", "Congo (Brazzaville)", "Congo (Kinshasa)"], 
                               ["Taiwan", "USA", "Republic of the Congo", "Democratic Republic of the Congo"],
                               inplace = True)
  #Set new index
  df.set_index('Country/Region', inplace = True)

  #Grouping together countries originally split into multiple Provinces/States
  #(Australia, Canada, China, Denmark, France, Netherlands, United Kingdom)
  df = df.groupby(by = df.index, axis = 0).sum()

  # Addition of a "global" row
  df.loc['Global'] = df.sum(axis=0)

  #transpose
  df=df.T
  df.rename_axis(index='Date', columns="Country",inplace=True)
  df.index = pd.to_datetime(df.index)

  #Naming the Date index:
  df.index.rename("Date", inplace= True)

  #Creating a datetime index:
  df.index = pd.to_datetime(df.index)
  
  #Save new dataFrame
  all_dataframes[key] = df

# Change country names in df_countries
old_names = [ 'Myanmar [Burma]', 'Cape Verde',  "C?te d'Ivoire", 'Czech Republic',
             'Congo [DRC]',  'Swaziland', 'Vatican City',  'South Korea', 'Macedonia [FYROM]',
              'Congo [Republic]',  'S?o Tom? and Pr?ncipe',  'United States',  'Gaza Strip','Sudan']

new_names = ['Burma', 'Cabo Verde', "Cote d'Ivoire", 'Czechia', 'Democratic Republic of the Congo',
              'Eswatini',  'Holy See',  'Korea, South',  'North Macedonia',  'Republic of the Congo',
             'Sao Tome and Principe', 'USA',  'West Bank and Gaza','South Sudan']

all_dataframes['countries'] = all_dataframes['countries'].replace(old_names, new_names)

cleaning: confirmed
cleaning: death
cleaning: recovered
cleaning: countries


In [60]:
all_dataframes['confirmed']
#now 186 countries in total + cruise ships "Diamond Princess" & "MS Zaandam" + "Global" count

Country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,Comoros,Costa Rica,...,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,USA,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe,Global
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,548,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,555
2020-01-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,643,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,654
2020-01-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,920,0,0,0,...,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,5,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,941
2020-01-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1406,0,0,0,...,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,7,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,1434
2020-01-26,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2075,0,0,0,...,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,8,0,0,0,0,0,5,0,0,0,0,0,0,0,2,0,0,0,0,0,2118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-24,10582,998,8306,762,69,25,12076,6661,7114,16503,4122,100,9138,33610,92,36198,57092,18,191,24,6263,2401,35,363211,141,2427,814,201,42,380,124,4890,86106,604,675,69102,84095,21175,87,930,...,11159,11,707,31616,1509,1468,1594,22583,655,235772,1141,3820,11,33459,30736,86,441,2929,509,3040,24,381,116,1051,156827,1643246,198,20986,29485,260916,769,3164,1121,325,423,9,222,920,56,5407613
2020-05-25,11173,1004,8503,763,70,25,12628,7113,7126,16539,4271,100,9171,35585,92,37144,57342,18,191,27,6660,2406,35,374898,141,2433,832,203,42,390,124,4890,87119,652,687,73997,84102,21981,87,951,...,11193,11,735,31960,1511,1469,1689,23615,806,235400,1182,3976,11,33843,30746,106,441,3100,509,3042,24,386,116,1051,157814,1662302,222,21245,30307,262547,787,3189,1177,326,423,9,233,920,56,5495061
2020-05-26,11831,1029,8697,763,70,25,13228,7402,7139,16557,4403,100,9366,36751,92,38059,57455,18,208,27,7136,2416,35,391222,141,2443,832,206,42,390,124,5436,88090,671,700,77961,84103,23003,87,956,...,11227,11,754,32343,1513,1469,1711,24264,806,236259,1319,3976,11,34440,30761,121,441,3266,509,3045,24,391,116,1051,158762,1680913,253,21584,31086,266599,789,3290,1211,327,429,9,249,920,56,5589626


### New Dataframe: Active Cases

In [0]:
all_dataframes['active'] = all_dataframes['confirmed'] - all_dataframes['recovered'] - all_dataframes['death']

## Calculating Projections (Marco)

In [0]:
def create_country(country):
  country_confirmed=all_dataframes["confirmed"].loc[:,[country]]
  country_confirmed["Total Cases"]=country_confirmed[country]
  country_confirmed.drop(country, inplace=True, axis=1)
  country_confirmed.rename_axis(columns=country,inplace=True)
  country_confirmed["New Cases"]=(country_confirmed["Total Cases"]-country_confirmed.shift(1)["Total Cases"])
  country_confirmed["R_7"]=(country_confirmed["Total Cases"]-country_confirmed.shift(7)["Total Cases"])/(country_confirmed.shift(4)["Total Cases"]-country_confirmed.shift(11)["Total Cases"])
  return(country_confirmed)

country_dataframes = {country: create_country(country) for country in all_dataframes["confirmed"].columns}

In [63]:
country_dataframes["Argentina"].tail(10)

Argentina,Total Cases,New Cases,R_7
Date,,,
2020-05-19,8809,438.0,1.202355
2020-05-20,9283,474.0,1.184820
2020-05-21,9931,648.0,1.375123
2020-05-22,10649,718.0,1.514572
2020-05-23,11353,704.0,1.579697
2020-05-24,12076,723.0,1.667221
2020-05-25,12628,552.0,1.521988
2020-05-26,13228,600.0,1.394006
2020-05-27,13933,705.0,1.310598


In [0]:
from datetime import timedelta

def create_projection(country):

  country_prediction=(pd.DataFrame(index=pd.date_range(start=country_dataframes[country].index[-1], periods=14, freq='D'), 
                                   columns=["Total Cases", "New Cases", "R_7"]))
  country_prediction.drop(country_prediction.index[0], inplace=True)
  country_prediction.rename_axis(index='Date', columns=country,inplace=True)
  country_prediction.index=pd.to_datetime(country_prediction.index)
  country_dataframes[country]["Prediction"]=False
  country_prediction["Prediction"]=True

  country_combined=pd.concat([country_dataframes[country], country_prediction])

  R_slidervalue_country=country_dataframes[country]["R_7"][-1]

  for date in country_combined.index:
    if country_combined.loc[date, 'Prediction']:
      country_combined.loc[date, 'Total Cases']=R_slidervalue_country*(country_combined.shift(4).loc[date, 'Total Cases']
                                                                       -country_combined.shift(5).loc[date, 'Total Cases'])+country_combined.shift(1).loc[date, 'Total Cases']

  country_combined['New Cases']=country_combined['Total Cases']-country_combined.shift(1)['Total Cases']
  for date in country_combined.index:
    try:
      country_combined.loc[date, "R_7"]=(country_combined.loc[date, "Total Cases"]-country_combined.shift(7).loc[date, "Total Cases"])/(country_combined.shift(4).loc[date, "Total Cases"]-country_combined.shift(11).loc[date, "Total Cases"])
    except ZeroDivisionError:
      country_combined.loc[date, 'R_7']=None
  country_combined["Total Cases"]=country_combined["Total Cases"].astype('float64')
  country_combined["New Cases"]=country_combined["New Cases"].astype('float64')
  country_combined["R_7"]=country_combined["R_7"].astype('float64')
  return (country_combined)  
projection_dataframes = {country: create_projection(country) for country in all_dataframes["confirmed"].columns}

In [0]:
pd.set_option('display.max_rows', 100)

In [78]:
projection_dataframes['Germany']['Death Cases']=all_dataframes['death']['Germany']
projection_dataframes['Germany']['Recovered Cases']=all_dataframes['recovered']['Germany']
projection_dataframes['Germany']['Active Cases']=(projection_dataframes['Germany']['Total Cases']
                                                  -projection_dataframes['Germany']['Death Cases']-projection_dataframes['Germany']['Recovered Cases'])
projection_dataframes['Germany']['Recovered Rate']=((projection_dataframes['Germany']['Recovered Cases']-projection_dataframes['Germany']['Recovered Cases'].shift(1))/
                                                    projection_dataframes['Germany']['Active Cases'].shift(1))
projection_dataframes['Germany']['Death Rate']=((projection_dataframes['Germany']['Death Cases']-projection_dataframes['Germany']['Death Cases'].shift(1))/
                                                projection_dataframes['Germany']['New Cases'].shift(21))

Recovered_Rate=projection_dataframes['Germany']['Recovered Rate'].loc[projection_dataframes['Germany']['Prediction']==False].iloc[-8:].mean()
Death_Rate=projection_dataframes['Germany']['Death Rate'].loc[projection_dataframes['Germany']['Prediction']==False].iloc[-8:].mean()

for date in projection_dataframes['Germany'].index:
    if projection_dataframes['Germany'].loc[date, 'Prediction']:
      projection_dataframes['Germany'].loc[date, 'Recovered Cases']=(Recovered_Rate*projection_dataframes['Germany'].shift(1).loc[date, 'Active Cases']
                                                                     +projection_dataframes['Germany'].shift(1).loc[date, 'Recovered Cases'])
      projection_dataframes['Germany'].loc[date, 'Death Cases']=(Death_Rate*projection_dataframes['Germany'].shift(21).loc[date, 'New Cases']
                                                                 +projection_dataframes['Germany'].shift(1).loc[date, 'Death Cases'])
      projection_dataframes['Germany'].loc[date, 'Active Cases']=(projection_dataframes['Germany'].loc[date, 'Total Cases']
                                                                  -projection_dataframes['Germany'].loc[date, 'Recovered Cases']
                                                                  -projection_dataframes['Germany'].loc[date, 'Death Cases'])
      
projection_dataframes['Germany']['Recovered Rate']=((projection_dataframes['Germany']['Recovered Cases']-projection_dataframes['Germany']['Recovered Cases'].shift(1))/
                                                    projection_dataframes['Germany']['Active Cases'].shift(1))
projection_dataframes['Germany']['Death Rate']=((projection_dataframes['Germany']['Death Cases']-projection_dataframes['Germany']['Death Cases'].shift(1))/
                                                projection_dataframes['Germany']['New Cases'].shift(21))  
                                                                 
projection_dataframes['Germany'].tail(20)



Germany,Total Cases,New Cases,R_7,Prediction,Death Cases,Recovered Cases,Active Cases,Recovered Rate,Death Rate
Date,,,,,,,,,
2020-05-22,179710.000000,689.000000,1.126289,False,8228.000000,159064.000000,12418.000000,0.076742,0.023408
2020-05-23,179986.000000,276.000000,0.919036,False,8261.000000,159716.000000,12009.000000,0.052504,0.037079
2020-05-24,180328.000000,342.000000,0.904914,False,8283.000000,160281.000000,11764.000000,0.047048,0.031564
2020-05-25,180600.000000,272.000000,0.891261,False,8309.000000,161199.000000,11092.000000,0.078035,0.053279
2020-05-26,181200.000000,600.000000,0.764351,False,8372.000000,161967.000000,10861.000000,0.069239,0.073684
2020-05-27,181524.000000,324.000000,0.720595,False,8428.000000,162820.000000,10276.000000,0.078538,0.048485
2020-05-28,182196.000000,672.000000,0.801970,False,8470.000000,163360.000000,10366.000000,0.052550,0.033123
2020-05-29,182414.135893,218.135893,0.667853,True,8519.324658,164057.818294,9836.992941,0.067318,0.042595
2020-05-30,182895.318010,481.182117,0.850181,True,8550.674354,164720.024896,9624.618760,0.067318,0.042595


In [0]:
def add_more_projections(country):
  projection_dataframes[country]['Death Cases']=all_dataframes['death'][country]
  projection_dataframes[country]['Recovered Cases']=all_dataframes['recovered'][country]
  projection_dataframes[country]['Active Cases']=(projection_dataframes[country]['Total Cases']
                                                    -projection_dataframes[country]['Death Cases']-projection_dataframes[country]['Recovered Cases'])
  projection_dataframes[country]['Recovered Rate']=((projection_dataframes[country]['Recovered Cases']-projection_dataframes[country]['Recovered Cases'].shift(1))/
                                                      projection_dataframes[country]['Active Cases'].shift(1))
  projection_dataframes[country]['Death Rate']=((projection_dataframes[country]['Death Cases']-projection_dataframes[country]['Death Cases'].shift(1))/
                                                  projection_dataframes[country]['New Cases'].shift(21))

  Recovered_Rate=projection_dataframes[country]['Recovered Rate'].loc[projection_dataframes[country]['Prediction']==False].iloc[-8:].mean()
  Death_Rate=projection_dataframes[country]['Death Rate'].loc[projection_dataframes[country]['Prediction']==False].iloc[-8:].mean()

  for date in projection_dataframes[country].index:
      if projection_dataframes[country].loc[date, 'Prediction']:
        projection_dataframes[country].loc[date, 'Recovered Cases']=(Recovered_Rate*projection_dataframes[country].shift(1).loc[date, 'Active Cases']
                                                                       +projection_dataframes[country].shift(1).loc[date, 'Recovered Cases'])
        projection_dataframes[country].loc[date, 'Death Cases']=(Death_Rate*projection_dataframes[country].shift(21).loc[date, 'New Cases']
                                                                   +projection_dataframes[country].shift(1).loc[date, 'Death Cases'])
        projection_dataframes[country].loc[date, 'Active Cases']=(projection_dataframes[country].loc[date, 'Total Cases']
                                                                    -projection_dataframes[country].loc[date, 'Recovered Cases']
                                                                    -projection_dataframes[country].loc[date, 'Death Cases'])
      
  projection_dataframes[country]['Recovered Rate']=((projection_dataframes[country]['Recovered Cases']-projection_dataframes[country]['Recovered Cases'].shift(1))/
                                                      projection_dataframes[country]['Active Cases'].shift(1))
  projection_dataframes[country]['Death Rate']=((projection_dataframes[country]['Death Cases']-projection_dataframes[country]['Death Cases'].shift(1))/
                                                  projection_dataframes[country]['New Cases'].shift(21))  
                                                                 
for country in all_dataframes["confirmed"].columns:
  add_more_projections(country)

In [90]:
projection_dataframes['Brazil'].tail(20)

Brazil,Total Cases,New Cases,R_7,Prediction,Death Cases,Recovered Cases,Active Cases,Recovered Rate,Death Rate
Date,,,,,,,,,
2020-05-22,330890.000000,20803.000000,1.289423,False,21048.000000,135430.000000,174412.000000,0.057716,0.199601
2020-05-23,347398.000000,16508.000000,1.215819,False,22013.000000,142587.000000,182798.000000,0.041035,0.197019
2020-05-24,363211.000000,15813.000000,1.203949,False,22666.000000,149911.000000,190634.000000,0.040066,0.138172
2020-05-25,374898.000000,11687.000000,1.117918,False,23473.000000,153833.000000,197592.000000,0.020573,0.118781
2020-05-26,391222.000000,16324.000000,1.079006,False,24512.000000,158593.000000,208117.000000,0.024090,0.152012
2020-05-27,411821.000000,20599.000000,1.055801,False,25598.000000,166647.000000,219576.000000,0.038699,0.097347
2020-05-28,438238.000000,26417.000000,1.049291,False,26754.000000,177604.000000,233880.000000,0.049901,0.126173
2020-05-29,450501.067829,12263.067829,1.000678,True,28404.722004,187296.405334,234799.940491,0.041442,0.148433
2020-05-30,467629.699577,17128.631748,1.007497,True,29765.406169,197026.934647,240837.358761,0.041442,0.148433


In [0]:
                   from matplotlib import pyplot as plt
from matplotlib import dates as mpl_dates
plt.xkcd()
fig, ax = plt.subplots()

country='Brazil'
variable='Total Cases'

ax.plot_date(projection_dataframes[country].loc[projection_dataframes[country]['Prediction']==False]
             .index, projection_dataframes[country].loc[projection_dataframes[country]['Prediction']==False][variable], label="Data", linestyle='solid', marker=None)
ax.plot_date(projection_dataframes[country].loc[projection_dataframes[country]['Prediction']==True]
             .index, projection_dataframes[country].loc[projection_dataframes[country]['Prediction']==True][variable], label="Projection", linestyle='--', marker=None)
date_format=mpl_dates.DateFormatter('%b, %d %Y')
ax.xaxis.set_major_formatter(date_format)
ax.set_xlabel('Date')
ax.set_ylabel('Total Cases')
ax.set_title('Brazil (Total Cases)')
ax.legend()
fig.autofmt_xdate()

plt.show()

# Data Visualization

## Static Data Visualization (Matplotlib)

In [0]:
 # temporarily markdown

for i in df_confirmed:
  print("The country is:", i)
  plt.figure()
  plt.plot(df_confirmed.index, df_confirmed[i], color = "b", label = "Confirmed Cases");
  plt.plot(df_active.index, df_active[i], color = "y", label = "Active Cases");
  plt.plot(df_death.index, df_death[i], color = "r", label = "Death Cases");
  plt.plot(df_recovered.index, df_recovered[i], color = "g", label = "Recovered Cases");
  plt.legend(loc = "best")
  plt.show()

## Interactive Data Visualization <br>


### World Map (temporarily deleted)


### Interactive Plotting

In [0]:
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool

import warnings
warnings.filterwarnings('ignore')

output_notebook()

# data
x_line = all_dataframes['active'].index
y_line = all_dataframes['active'].Germany

# line plot
line_plot = figure(plot_width=500, plot_height=325, title='Covid-19 Cases in Germany', x_axis_label='Date', 
                   y_axis_label='Active Cases', x_axis_type='datetime')
line_plot.line(x_line, y_line, legend='Active Cases', line_width=2)


show(line_plot)

#### Using Bokeh and ipywidgets

In [0]:
# line plot - multiple lines

from bokeh.layouts import layout
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from bokeh.models import Range1d, Span

############################################################################
#Widgets:

#Dropdown Widget:
drop_down = widgets.Dropdown(options=all_dataframes['confirmed'].columns,
                                value="Germany",
                                description='Country:',
                                disabled=False)

#Selection Range Slider Widget:
range_slider= widgets.SelectionRangeSlider(
    options=all_dataframes["confirmed"].index,
    index=(0, 124),
    description='Timeframe',
    disabled=False
)

# create slider
#dates = list(pd.date_range(all_dataframes["confirmed"].index.min(), all_dataframes["confirmed"].index.max(), freq='D'))
#options = [(i.strftime('%Y-%m-%d'), i) for i in dates]
#index = (0, len(dates)-1)
#myslider = widgets.SelectionRangeSlider(
#    options = options,
#    index = index,
#    description = 'Timeframe:',
#    orientation = 'horizontal',
#    layout={'width': '800px'}
#)

startdatepicker = widgets.DatePicker(
    description='Pick a Startdate:',
    disabled=False,
    style = {'description_width': 'initial'}
)

enddatepicker = widgets.DatePicker(
    description='Pick an Enddate:',
    disabled=False,
    style = {'description_width': 'initial'}
)

#Multi-Select Widget:

#multi_select = widgets.SelectMultiple(options=all_dataframes.keys(),
#    value=['confirmed'],
#    #rows=10,
#    description='Counts:',
#    disabled=False
#)
############################################################################

############################################################################
#plotting function
def plot_country(country, startdate, enddate,
                 confirmed = True, active = False, recovered = False, death = False,
                 include_prediction = False
                ):

  # plot: 
  multi_line_plot = figure(plot_width=900, plot_height=425, title='Covid-19 Cases in {}'.format(country), 
                           x_axis_label='Date', y_axis_label='No. of Cases in {}'.format(country),
                           x_axis_type='datetime', y_axis_type = "linear")
  multi_line_plot.title.text_font_size = "35px"
  multi_line_plot.title.align = "center"

  multi_line_plot.left[0].formatter.use_scientific = False
    
  #lines:
  if confirmed == True:
      multi_line_plot.line(x = "Date", y=country, source = all_dataframes['confirmed'],
                       color='black', line_width=2, line_dash="dashed", legend = "Confirmed Cases")
  if active == True:
      multi_line_plot.line(x = "Date", y=country, source = all_dataframes['active'],
                       color='blue', line_width=2, legend = "Active Cases")
  if recovered == True:
      multi_line_plot.line(x = "Date", y=country, source = all_dataframes['recovered'],
                       color='green', line_width=2, legend = "Recovered Cases")
  if death == True:
      multi_line_plot.line(x = "Date", y=country, source = all_dataframes['death'],
                       color='red', line_width=2, legend = "Death Cases")

  if include_prediction == True:
    multi_line_plot.line(projection_dataframes[country].loc[projection_dataframes[country]['Prediction']==True]
             .index, projection_dataframes[country].loc[projection_dataframes[country]['Prediction']==True][variable], 
                 legend="Projection", line_dash = "dashed",  color='black', line_width=2)    
  multi_line_plot.x_range = Range1d(startdate, enddate)
  
    
  prediction_beginning = all_dataframes['confirmed'].index.max()
  beginning_of_prediction = Span(location=prediction_beginning,
                            dimension='height', line_color='red',
                            line_dash='dashed', line_width=1) 
  multi_line_plot.add_layout(beginning_of_prediction)
    
  # add hover tool
  hover = HoverTool()
  hover.tooltips = [('Date','@Date{%F}'),('No. of Cases','@{}'.format(country))]
  hover.formatters = {'Date': 'datetime'} 
  multi_line_plot.add_tools(hover)

  #format legend
  multi_line_plot.legend.location = "top_left"
  multi_line_plot.legend.orientation = "vertical"
  multi_line_plot.legend.background_fill_alpha = 0.35

  show(multi_line_plot)
############################################################################

############################################################################
#Interact function:
interact(plot_country, country = drop_down, startdate = startdatepicker, enddate = enddatepicker);
############################################################################


#To complete the plot, we need to implement the forecast

#We not only need a forecast for the confirmed cases (confirmed cases from day before + new cases),
#but also a forecast for the active cases, the death cases and the recovered cases!

#Then we can think about implementing the forecasts into the plot, maybe even with an additional checkbox
#e.g "Include forecasts?" 
#and an easy if-clause in the plot_country function

#### Using Bokeh with its own dropdown widget

In [0]:
# temporarily markdown

#different approach: Here, I try to use bokeh's own dropdown widget instead of the ipywidgets dropdown widget

from bokeh.io import output_file, show, push_notebook
from bokeh.models import Dropdown
from bokeh.layouts import column, row

menu = list(df_confirmed.columns)
dropdown = Dropdown(label="Country Selection", button_type="warning", menu=menu, value="Germany")

#dropdown.js_link("value", multi_line_plot, "country")
#here I have to write a javascript link to link a change in the dropdown to a change in the graph, which I can't do

show(row(line_plot, dropdown))

In [0]:
# temporarily markdown

#different idea was to use a stacked area plot instead of a line plot
x = df_confirmed.index
stacked_area_plot = figure(plot_width=600, plot_height=300, x_axis_type='datetime')

stacked_area_plot.varea_stack(x=x, 
                              stackers= [df_death.Germany, df_recovered.Germany, df_active.Germany],
                              color=('red', 'green', 'orange'))

show(stacked_area_plot)

In [0]:
# stacked_area_plot = figure(plot_width=600, plot_height=300)
# stacked_area_plot.varea(y1=stack(),       y2=stack('Germany'),         x='Date', color='red', source=df_death, name='Death Cases')
# stacked_area_plot.varea(y1=stack('2016'), y2=stack('2016', '2017'), x='Date', color='green',  source=df_recovered, name='Recovered Cases')

### using ipywidgets

In [0]:
#!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
%matplotlib inline
import matplotlib.dates as mdates

#dstart = df_confirmed.index[startdate]
#dend = df_confirmed.index[enddate]

def country_visualization(country):#, startdate, enddate):
  #plt.figure()
  fig, ax = plt.subplots(figsize = [18.5, 6.5], dpi = 200, facecolor = "WhiteSmoke")
  ax.plot(all_dataframes['confirmed'].index, all_dataframes['confirmed'][country], color = "black", linestyle = "dashed", linewidth = 2, label = "Confirmed Cases");
  #ax.set_xlim([pd.to_datetime('2020-01-28 23:50:00'), pd.to_datetime('2020-02-29 00:10:00')])
  #plt.xlim(pd.to_datetime('2020-01-28 23:50:00'), pd.to_datetime('2020-02-29 00:10:00'))
  #plt.xlim(xlim((left, right)))
  ax.set_xticks(all_dataframes['confirmed'].index)
  ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
  ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m-%d"))
  _=plt.xticks(rotation=90)
  #plt.xlim(df_confirmed.index[startdate], df_confirmed.index[enddate])
  #plt.xlim([df_confirmed.index[startdate], df_confirmed.index[enddate]])
  ax.plot(all_dataframes['active'].index, all_dataframes['active'][country], color = "b", linestyle = "-", linewidth = 2, label = "Active Cases");
  #ax.set_xlim([pd.to_datetime('2020-01-28 23:50:00'), pd.to_datetime('2020-02-29 00:10:00')])
  ax.plot(all_dataframes['death'].index, all_dataframes['death'][country], color = "r", linestyle = "-", linewidth = 2, label = "Death Cases");
  #ax.set_xlim([pd.to_datetime('2020-01-28 23:50:00'), pd.to_datetime('2020-02-29 00:10:00')])
  ax.plot(all_dataframes['recovered'].index, all_dataframes['recovered'][country], color = "g", linestyle = "-", linewidth = 2, label = "Recovered Cases");
  #ax.set_xlim([pd.to_datetime('2020-01-28 23:50:00'), pd.to_datetime('2020-02-29 00:10:00')])
  #plt.xlim(pd.to_datetime('2020-01-28 23:50:00'), pd.to_datetime('2020-02-29 00:10:00'))
  ax.legend(loc = "best");
  plt.xlabel('Date')
  plt.ylabel('No. of Cases')
  plt.title('Covid-19 Cases in {}'.format(country))
  #plt.set_xlim([df_confirmed.index[startdate], df_confirmed.index[enddate]])
  plt.show();

interactive_plot = interact(country_visualization, country = all_dataframes['confirmed'].columns)#, startdate = all_dataframes['confirmed'].index, enddate = all_dataframes['confirmed'].index)
interactive_plot;

In [0]:
import datetime
dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b'), i) for i in dates]
widgets.SelectionRangeSlider(
    options=options,
    index=(0, 11),
    description='Months (2015)',
    disabled=False
)